In [1]:
import numpy as np
np.random.seed(10)
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline


from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
names=['id','id_str','screen_name','location','description','url','followers_count','friends_count','listedcount','created_at','favourites_count','verified','statuses_count','lang','status','default_profile','default_profile_image','has_extended_profile','name','bot']

In [3]:
#bots_data = pd.read_csv('/Users/Heng2j/Documents/NYU Grad /Spring 2017/Machine Learning/project/bots_data.csv',encoding="ISO-8859-1",header = None, na_values='?',delim_whitespace=False,)
#bots_data = bots_data.ix[1:]

#nonbots_data = pd.read_csv('/Users/Heng2j/Documents/NYU Grad /Spring 2017/Machine Learning/project/nonbots_data.csv',encoding="ISO-8859-1",header = None,na_values='?',delim_whitespace=False,)
#nonbots_data = nonbots_data.ix[1:]


#data_frames = [bots_data, nonbots_data]

#df = pd.concat(data_frames)

df = pd.read_csv('merged.csv',encoding="ISO-8859-1",
                 header=None,delim_whitespace=False,names=names,na_values='?',)
df.columns = names


In [4]:
df.verified=df.verified.astype('bool')
df.verified=df.verified.astype(int)
df.default_profile=df.default_profile.astype('bool')
df.default_profile=df.default_profile.astype(int)
df.default_profile_image=df.default_profile_image.astype('bool')
df.default_profile_image=df.default_profile_image.astype(int)


df.followers_count = df.followers_count.astype(int)
df.friends_count = df.friends_count.astype(int)
df.listedcount = df.listedcount.astype(int)
df.favourites_count = df.favourites_count.astype(int)
df.statuses_count = df.statuses_count.astype(int)
df.bot = df.bot.astype(int)

In [5]:
# Creating more features from given dataframe
df["screen_name_len"] = [len(i) for i in df["screen_name"]]
df["bot_is_substr"] = [int('bot' in i.lower()) for i in df["screen_name"]]
df["bot_in_des"] = [int('bot' in str(i).lower()) for i in df['description']]

# Getting the ages in years from created_at
ages = []
for i in df["created_at"]:
    if len(i) > 20:
        i = i.split()
        year = int(i[5][:4]) % 2000 
    else:
        i = i.split('/')
        year = int(i[2].split()[0]) % 2000
    age = 17-year
    ages.append(age)
df["age"] = ages

# Getting the edit distance between name and screen_name

df.name = df.name.astype(str)
df.screen_name = df.screen_name.astype(str)

# edit_distance function taken from stack overflow 
def edit_distance(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

def edit_dis_list(l1, l2):
    ed = []
    for i in range(len(l1)):
        ed.append(edit_distance(l1[i].lower(),l2[i].lower()))
    return ed

df["edit_dis"] = edit_dis_list(df["screen_name"],df["name"]) 

# gender 
from genderize import Genderize

male = [0] * 2231
female = [0] * 2231
i = 0

names = list(df.name.astype(str))

for name in names:
    print(name)
    name = name.split()
    try:
        gInfo = Genderize().get(name)
        m = 1
        f = 1
        for item in gInfo:
            if item['gender'] == 'male':
                male[i] += item['probability']
                m += 1
            elif item['gender'] == 'female':
                female[i] += item['probability']
                f += 1
        male[i] /= m
        female[i] /= f
    except Exception:
        pass
    
    i += 1

df['male_prob'] = male
df['fem_prob'] = female
        
            
        
#df.head()

McGucket bot
MJ Kuhn
duckNote
robot recipes
ÕvÕrÕ Õ_mlÕ_Õ_ö_
GLOSSATORY
Fancypants
YOU the Hedgehog
Jamie Phillips
Donella Nothem
Ebooks Brilhantes
__ image bot convos __
Kindra Stobbe
cga.graphics
Helen Nguyen
Michael Nishimura
He's-a-back!
litpatches_txt
Every Simile
Jeff Christensen
"Far-West"
Dr. Phil Notademon
Divina Cooper
Library of Emoji
PS & WRAP
Fake Bloomberg News
Oliver Twitch
Brock Jargotron
Games 4 Grownups
Lowpoly Bot
2blessed.exe
poem.exe
EdSurge HigherEd
GUNNERCARTERXX
Mohamed Abd El-Rhman
Star Near You
wayback_exe
wutahiq19
LSystemBot 1.0
Low Battery
yeahashleyrober
Wonder Woman
Game Walkthroughs
Netflix Genres
Altha Feinman
Voynich Tech News
READ BOI XX5SOSVINES
throam bot
"Pokemon"
ButtFeed
Gilgamesh Bot
dronesweetie
Penry Parmelee
Altagracia Sanders
Book On Balcony
Superb Owl
a river
tiny astronaut
ceubehen
Lillian Warner
kpop
marcelinep16
epigenetics_papers
TardyGram
Space Sloth
LucileRobertsa
Pussy-kun
Advent V. Recall
Great Artist
Nerd@Justin fan
patrinafb

In [9]:
# This cell is for adding polarity and subjectivity as features 
from textblob import TextBlob

descriptions = [TextBlob(str(txt)) for txt in df['description']]

# Creating lists of the polarity and the descriptions
desc_pol = [blob.sentiment.polarity for blob in descriptions]
desc_subj = [blob.sentiment.subjectivity for blob in descriptions]

# Turning them into features
df["desc_pol"] = desc_pol
df["desc_subj"] = desc_subj

In [10]:
#Reindex the merged dataframe df
df.reset_index(drop=True, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2231 entries, 0 to 2230
Data columns (total 29 columns):
id                       2231 non-null float64
id_str                   2231 non-null object
screen_name              2231 non-null object
location                 1419 non-null object
description              1910 non-null object
url                      1212 non-null object
followers_count          2231 non-null int64
friends_count            2231 non-null int64
listedcount              2231 non-null int64
created_at               2231 non-null object
favourites_count         2231 non-null int64
verified                 2231 non-null int64
statuses_count           2231 non-null int64
lang                     2231 non-null object
status                   2202 non-null object
default_profile          2231 non-null int64
default_profile_image    2231 non-null int64
has_extended_profile     2153 non-null object
name                     2231 non-null object
bot                      2

In [21]:
df.head()

,followers_count,friends_count,listedcount,favourites_count,verified,statuses_count,default_profile,default_profile_image,bot,screen_name_len,bot_is_substr,bot_in_des,age,edit_dis,male_prob,fem_prob,desc_pol,desc_subj
0,1129,7,2,0,0,23557,0,0,1,12,1,1,2,1,0,0,0.0,0.0
1,0,22,0,0,0,1,1,1,1,8,0,0,0,8,0,0,0.0,0.0
2,3,0,3,0,0,1050,1,0,1,13,0,0,3,5,0,0,0.0,0.0
3,505,13,49,0,0,5109,0,0,1,12,1,1,2,1,0,0,0.0,0.1
4,15,0,11,0,0,31365,1,0,1,11,0,1,2,16,0,0,0.0,0.0


In [22]:
del df['id']
del df['screen_name']
del df ['id_str']
del df['location']
del df['url']
del df['name']
del df['description']
del df['created_at']
del df['status']
del df['lang']
del df['has_extended_profile']

KeyError: 'id'

In [23]:
df.head()

,followers_count,friends_count,listedcount,favourites_count,verified,statuses_count,default_profile,default_profile_image,bot,screen_name_len,bot_is_substr,bot_in_des,age,edit_dis,male_prob,fem_prob,desc_pol,desc_subj
0,1129,7,2,0,0,23557,0,0,1,12,1,1,2,1,0,0,0.0,0.0
1,0,22,0,0,0,1,1,1,1,8,0,0,0,8,0,0,0.0,0.0
2,3,0,3,0,0,1050,1,0,1,13,0,0,3,5,0,0,0.0,0.0
3,505,13,49,0,0,5109,0,0,1,12,1,1,2,1,0,0,0.0,0.1
4,15,0,11,0,0,31365,1,0,1,11,0,1,2,16,0,0,0.0,0.0


In [24]:
df.head()

,followers_count,friends_count,listedcount,favourites_count,verified,statuses_count,default_profile,default_profile_image,bot,screen_name_len,bot_is_substr,bot_in_des,age,edit_dis,male_prob,fem_prob,desc_pol,desc_subj
0,1129,7,2,0,0,23557,0,0,1,12,1,1,2,1,0,0,0.0,0.0
1,0,22,0,0,0,1,1,1,1,8,0,0,0,8,0,0,0.0,0.0
2,3,0,3,0,0,1050,1,0,1,13,0,0,3,5,0,0,0.0,0.0
3,505,13,49,0,0,5109,0,0,1,12,1,1,2,1,0,0,0.0,0.1
4,15,0,11,0,0,31365,1,0,1,11,0,1,2,16,0,0,0.0,0.0


In [25]:
for a in range(1000):
    randomized_Data = df.reindex(np.random.permutation(df.index))

train_df = randomized_Data[:int((len(randomized_Data)+1)*.80)]
test_df = randomized_Data[int((len(randomized_Data)+1)*.80):] 


In [26]:
#Create the Predicting Variables and Targeting Variables for Traning and Test sets
X_train = train_df.drop('bot', 1)
Y_train = train_df['bot']


X_test = test_df.drop('bot', 1)
Y_test = test_df['bot']

In [27]:
df.head()

,followers_count,friends_count,listedcount,favourites_count,verified,statuses_count,default_profile,default_profile_image,bot,screen_name_len,bot_is_substr,bot_in_des,age,edit_dis,male_prob,fem_prob,desc_pol,desc_subj
0,1129,7,2,0,0,23557,0,0,1,12,1,1,2,1,0,0,0.0,0.0
1,0,22,0,0,0,1,1,1,1,8,0,0,0,8,0,0,0.0,0.0
2,3,0,3,0,0,1050,1,0,1,13,0,0,3,5,0,0,0.0,0.0
3,505,13,49,0,0,5109,0,0,1,12,1,1,2,1,0,0,0.0,0.1
4,15,0,11,0,0,31365,1,0,1,11,0,1,2,16,0,0,0.0,0.0


In [29]:
# The two variables below aren't used
#rf_data = df.copy()
#Names=['age','screen_name_len', 'bot_is_substr','bot_in_des','followers_count','friends_count','listedcount', 'favourites_count','statuses_count','verified','statuses_count']
clf = RandomForestClassifier(n_estimators=700, oob_score=False, criterion = "entropy")
clf.fit(X_train, Y_train)
predicted = clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,predicted)

0.86771300448430488

# New Data


In [88]:
names=['id','id_str','screen_name','location','description','url','followers_count','friends_count','listedcount','created_at','favourites_count','verified','statuses_count','lang','status','default_profile','default_profile_image','has_extended_profile','name','bot']

In [89]:
# Store the paths here as strings 

training_data_path = 'training_data_2_csv_UTF.csv' #'/Users/Heng2j/Documents/NYU Grad /Spring 2017/Machine Learning/project/training_data_2_csv_UTF.csv'
testing_data_path = 'test_data_4_students.csv' #'/Users/Heng2j/Documents/NYU Grad /Spring 2017/Machine Learning/project/test_data_4_students.csv'

# Getting the csv's
training_data = pd.read_csv(training_data_path,encoding="ISO-8859-1",header = None, na_values='?',delim_whitespace=False,)
training_data = training_data.ix[1:]
training_data.columns = names


testing_data = pd.read_csv(testing_data_path,encoding="ISO-8859-1",header = None,na_values='?',delim_whitespace=False,)
testing_data = testing_data.ix[1:]
testing_data.columns = names

In [90]:
testing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 1 to 616
Data columns (total 20 columns):
id                       575 non-null object
id_str                   575 non-null object
screen_name              575 non-null object
location                 451 non-null object
description              559 non-null object
url                      463 non-null object
followers_count          575 non-null object
friends_count            575 non-null object
listedcount              575 non-null object
created_at               575 non-null object
favourites_count         575 non-null object
verified                 575 non-null object
statuses_count           575 non-null object
lang                     575 non-null object
status                   573 non-null object
default_profile          575 non-null object
default_profile_image    575 non-null object
has_extended_profile     575 non-null object
name                     575 non-null object
bot                      0 non-null obje

In [91]:
testing_data.tail()

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listedcount,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
mask = testing_data['id'] == 'NaN'

In [93]:
mask

1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
       ...  
587    False
588    False
589    False
590    False
591    False
592    False
593    False
594    False
595    False
596    False
597    False
598    False
599    False
600    False
601    False
602    False
603    False
604    False
605    False
606    False
607    False
608    False
609    False
610    False
611    False
612    False
613    False
614    False
615    False
616    False
Name: id, dtype: bool

In [94]:
testing_data_noNAN = testing_data[~mask]

In [95]:
testing_data_noNAN.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 616 entries, 1 to 616
Data columns (total 20 columns):
id                       575 non-null object
id_str                   575 non-null object
screen_name              575 non-null object
location                 451 non-null object
description              559 non-null object
url                      463 non-null object
followers_count          575 non-null object
friends_count            575 non-null object
listedcount              575 non-null object
created_at               575 non-null object
favourites_count         575 non-null object
verified                 575 non-null object
statuses_count           575 non-null object
lang                     575 non-null object
status                   573 non-null object
default_profile          575 non-null object
default_profile_image    575 non-null object
has_extended_profile     575 non-null object
name                     575 non-null object
bot                      0 non-null obje

In [96]:
testing_data_noNAN.tail()

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listedcount,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2797 entries, 1 to 2797
Data columns (total 20 columns):
id                       2797 non-null object
id_str                   2797 non-null object
screen_name              2797 non-null object
location                 1777 non-null object
description              2394 non-null object
url                      1501 non-null object
followers_count          2797 non-null object
friends_count            2797 non-null object
listedcount              2797 non-null object
created_at               2797 non-null object
favourites_count         2797 non-null object
verified                 2797 non-null object
statuses_count           2797 non-null object
lang                     2797 non-null object
status                   2763 non-null object
default_profile          2797 non-null object
default_profile_image    2797 non-null object
has_extended_profile     2698 non-null object
name                     2797 non-null object
bot                

In [98]:
training_data.verified=training_data.verified.astype('bool')
training_data.verified=training_data.verified.astype(int)
training_data.default_profile=training_data.default_profile.astype('bool')
training_data.default_profile=training_data.default_profile.astype(int)
training_data.default_profile_image=training_data.default_profile_image.astype('bool')
training_data.default_profile_image=training_data.default_profile_image.astype(int)


training_data.followers_count = training_data.followers_count.astype(int)
training_data.friends_count = training_data.friends_count.astype(int)
training_data.listedcount = training_data.listedcount.astype(int)
training_data.favourites_count = training_data.favourites_count.astype(int)
training_data.statuses_count = training_data.statuses_count.astype(int)
training_data.bot = training_data.bot.astype(int)

In [99]:
testing_data.verified=testing_data.verified.astype('bool')
testing_data.verified=testing_data.verified.astype(int)
testing_data.default_profile=testing_data.default_profile.astype('bool')
testing_data.default_profile=testing_data.default_profile.astype(int)
testing_data.default_profile_image=testing_data.default_profile_image.astype('bool')
testing_data.default_profile_image=testing_data.default_profile_image.astype(int)



In [100]:
testing_data.followers_count.replace('None', 0 , inplace=True)
testing_data.followers_count.replace(np.nan, 0 , inplace=True)

testing_data.friends_count.replace('None', 0 , inplace=True)
testing_data.friends_count.replace(np.nan, 0 , inplace=True)

testing_data.listedcount.replace('None', 0 , inplace=True)
testing_data.listedcount.replace(np.nan, 0 , inplace=True)


testing_data.favourites_count.replace('None', 0 , inplace=True)
testing_data.favourites_count.replace(np.nan, 0 , inplace=True)

testing_data.statuses_count.replace('None', 0 , inplace=True)
testing_data.statuses_count.replace(np.nan, 0 , inplace=True)

In [101]:

testing_data.followers_count = testing_data.followers_count.astype(int)
testing_data.friends_count = testing_data.friends_count.astype(int)
testing_data.listedcount = testing_data.listedcount.astype(int)
testing_data.favourites_count = testing_data.favourites_count.astype(int)
testing_data.statuses_count = testing_data.statuses_count.astype(int)
#testing_data.bot = testing_data.bot.astype(int)

In [105]:
# Creating more features from given dataframe
training_data["screen_name_len"] = [len(i) for i in training_data["screen_name"]]
training_data["bot_is_substr"] = [int('bot' in i.lower()) for i in training_data["screen_name"]]
training_data["bot_in_des"] = [int('bot' in str(i).lower()) for i in training_data['description']]



In [103]:
# Getting the ages in years from created_at
ages = []
for i in training_data["created_at"]:
    if len(i) > 20:
        i = i.split()
        year = int(i[5][:4]) % 2000 
    else:
        i = i.split('/')
        year = int(i[2].split()[0]) % 2000
    age = 17-year
    ages.append(age)
training_data["age"] = ages

In [108]:
# This cell is for adding polarity and subjectivity as features 
# For training data 
from textblob import TextBlob

descriptions = [TextBlob(str(txt)) for txt in training_data['description']]

# Creating lists of the polarity and the descriptions
desc_pol = [blob.sentiment.polarity for blob in descriptions]
desc_subj = [blob.sentiment.subjectivity for blob in descriptions]

# Turning them into features
training_data["desc_pol"] = desc_pol
training_data["desc_subj"] = desc_subj

# For testing data 
descriptions = [TextBlob(str(txt)) for txt in testing_data['description']]

# Creating lists of the polarity and the descriptions
desc_pol = [blob.sentiment.polarity for blob in descriptions]
desc_subj = [blob.sentiment.subjectivity for blob in descriptions]

# Turning them into features
testing_data["desc_pol"] = desc_pol
testing_data["desc_subj"] = desc_subj

In [109]:
training_data.head()

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listedcount,created_at,...,default_profile_image,has_extended_profile,name,bot,screen_name_len,bot_is_substr,bot_in_des,age,desc_pol,desc_subj
1,8.16E+17,"""815745789754417152""","""HoustonPokeMap""","""Houston, TX""","""Rare and strong PokÅ½mon in Houston, TX. See ...","""https://t.co/dnWuDbFRkt""",1291,0,10,"""Mon Jan 02 02:25:26 +0000 2017""",...,1,FALSE,"""Houston PokÅ½ Alert""",1,16,0,0,0,0.411111,0.711111
2,4843621225,4843621225,kernyeahx,"Templeville town, MD, USA",From late 2014 Socium Marketplace will make sh...,NaN,1,349,0,2/1/2016 7:37,...,1,FALSE,Keri Nelson,1,9,0,0,1,0.300000,0.542857
3,4303727112,4303727112,mattlieberisbot,NaN,"Inspired by the smart, funny folks at @replyal...",https://t.co/P1e1o0m4KC,1086,0,14,Fri Nov 20 18:53:22 +0000 2015,...,1,FALSE,Matt Lieber Is Bot,1,15,1,1,2,0.232143,0.821429
4,3063139353,3063139353,sc_papers,NaN,NaN,NaN,33,0,8,2/25/2015 20:11,...,1,FALSE,single cell papers,1,9,0,0,2,0.000000,0.000000
5,2955142070,2955142070,lucarivera16,"Dublin, United States",Inspiring cooks everywhere since 1956.,NaN,11,745,0,1/1/2015 17:44,...,1,FALSE,lucarivera16,1,12,0,0,2,0.500000,1.000000


In [110]:
testing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 1 to 616
Data columns (total 22 columns):
id                       575 non-null object
id_str                   575 non-null object
screen_name              575 non-null object
location                 451 non-null object
description              559 non-null object
url                      463 non-null object
followers_count          616 non-null int64
friends_count            616 non-null int64
listedcount              616 non-null int64
created_at               575 non-null object
favourites_count         616 non-null int64
verified                 616 non-null int64
statuses_count           616 non-null int64
lang                     575 non-null object
status                   573 non-null object
default_profile          616 non-null int64
default_profile_image    616 non-null int64
has_extended_profile     575 non-null object
name                     575 non-null object
bot                      0 non-null object
desc_

In [111]:
#testing_data["screen_name"]
testing_data.screen_name.replace(np.nan, 0 , inplace=True)
testing_data.screen_name = testing_data.screen_name.astype(str)

testing_data.description.replace(np.nan, 0 , inplace=True)
testing_data.description = testing_data.screen_name.astype(str)


In [112]:
# Creating more features from given dataframe

testing_data["screen_name_len"] = [len(i) for i in testing_data["screen_name"]]
testing_data["bot_is_substr"] = [int('bot' in i.lower()) for i in testing_data["screen_name"]]
testing_data["bot_in_des"] = [int('bot' in str(i).lower()) for i in testing_data['description']]



In [113]:
testing_data.created_at.replace(np.nan, 'Wed Apr 15 05:23:19 +0000 2017' , inplace=True)

In [114]:
# Getting the ages in years from created_at
ages = []
for i in testing_data["created_at"]:
    if len(i) > 20:
        i = i.split()
        year = int(i[5][:4]) % 2000 
    else:
        i = i.split('/')
        year = int(i[2].split()[0]) % 2000
    age = 17-year
    ages.append(age)
testing_data["age"] = ages





In [115]:
#Create the Predicting Variables and Targeting Variables for Traning and Test sets
X_train = training_data.drop('bot', 1)
Y_train = training_data['bot']


X_test = testing_data.drop('bot', 1)
Y_test = testing_data['bot']

In [116]:
# features are only the numerical and binary features 
features=['age','followers_count','friends_count','listedcount', 'verified', 'favourites_count','statuses_count','default_profile','default_profile_image','screen_name_len','bot_is_substr','bot_in_des']

features

['age',
 'followers_count',
 'friends_count',
 'listedcount',
 'verified',
 'favourites_count',
 'statuses_count',
 'default_profile',
 'default_profile_image',
 'screen_name_len',
 'bot_is_substr',
 'bot_in_des']

In [117]:
X_train = X_train[features]


X_test = X_test[features]

In [118]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 1 to 616
Data columns (total 12 columns):
age                      616 non-null int64
followers_count          616 non-null int64
friends_count            616 non-null int64
listedcount              616 non-null int64
verified                 616 non-null int64
favourites_count         616 non-null int64
statuses_count           616 non-null int64
default_profile          616 non-null int64
default_profile_image    616 non-null int64
screen_name_len          616 non-null int64
bot_is_substr            616 non-null int64
bot_in_des               616 non-null int64
dtypes: int64(12)
memory usage: 57.8 KB


In [119]:
from sklearn.ensemble import RandomForestClassifier
#rf_data = df.copy()
Names=['age','screen_name_len', 'bot_is_substr','bot_in_des','followers_count','friends_count','listedcount', 'favourites_count','statuses_count','verified','statuses_count']
clf = RandomForestClassifier(n_estimators=700, oob_score=False, criterion = "entropy")
clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=700, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [120]:
X_Test_predict = clf.predict(X_test)

In [121]:
X_Test_predict

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1,

In [122]:
testing_data['bot'] = X_Test_predict

In [123]:
testing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 1 to 616
Data columns (total 26 columns):
id                       575 non-null object
id_str                   575 non-null object
screen_name              616 non-null object
location                 451 non-null object
description              616 non-null object
url                      463 non-null object
followers_count          616 non-null int64
friends_count            616 non-null int64
listedcount              616 non-null int64
created_at               616 non-null object
favourites_count         616 non-null int64
verified                 616 non-null int64
statuses_count           616 non-null int64
lang                     575 non-null object
status                   573 non-null object
default_profile          616 non-null int64
default_profile_image    616 non-null int64
has_extended_profile     575 non-null object
name                     575 non-null object
bot                      616 non-null int64
desc

In [124]:
result = testing_data[['id','bot']]

In [59]:
result

,id,bot
1,2281292622,0
2,2344040251,0
3,765871267,0
4,4772373433,0
5,1324548560,1
6,2561341789,1
7,347810134,1
8,856303860,1
9,832875000000000000,1
10,88856792,0


In [125]:
result.to_csv('Result_20170503_02.csv',  sep=',', encoding='utf-8', index = False)